# __Auto-ML을 사용하여 분류 모델 만들기__

## __0. 데이터 준비__

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN={발급받은 개인 토큰}

In [2]:
%%thanosql
COPY titanic_train FROM "tutorial_data/titanic_data/titanic_train.csv"

Success


In [3]:
%%thanosql
COPY titanic_test FROM "tutorial_data/titanic_data/titanic_test.csv"

Success


## __1. 데이터 세트 확인__

생존자 예측 분류 모델을 만들기 위해 우리는 ThanoSQL DB에 저장되어 있는 titanic_train 테이블을 사용합니다. 아래의 쿼리문을 실행하면서 테이블 내용을 확인합니다.

In [4]:
%%thanosql
SELECT * 
FROM titanic_test 
LIMIT 5 

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## __2. 분류 모델 생성__

이전 단계에서 확인한 titanic_train 데이터를 사용하여 생존자 예측 분류 모델을 만듭니다. 아래의 쿼리 구문을 실행시켜 titanic_classification 이름의 모델을 만들어 봅니다.

In [7]:
%%thanosql
BUILD MODEL tutorial_automl_classification 
USING AutomlClassifier 
OPTIONS (
    target='survived', 
    impute_type='iterative',  
    features_to_drop=["name", 'ticket', 'passengerid', 'cabin'],
    time_left_for_this_task = 300
    ) 
AS 
SELECT * 
FROM titanic_train

Success


## __3. 생성된 모델 평가__

아래의 쿼리문을 실행하여 이전 단계에서 만든 예측 모델의 성능을 평가합니다. 

In [8]:
%%thanosql 
EVALUATE USING tutorial_automl_classification 
OPTIONS (
    target = 'survived'
    )
AS 
SELECT * 
FROM titanic_train

,metrics,value
0,accuracy,0.897868
1,ROCAUC,0.895374
2,Recall,0.886154
3,Precision,0.842105
4,f1-score,0.863568
5,Kappa,0.782039
6,MCC,0.782688


## __4. 생성된 모델을 사용하여 생존자 예측__

이전 단계에서 생성한 생존자 예측 모델을 사용해 탑승 승객 정보에 따른 생존 여부를 예측해 봅니다. 테스트용 데이터 세트(학습에 이용되지 않은 데이터 테이블, titanic_test)를 사용합니다.

In [9]:
%%thanosql 
PREDICT USING tutorial_automl_classification
AS 
SELECT * 
FROM titanic_test

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,predicted
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,None,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,None,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,None,S,0
